## Data Cleaning

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("SparkSession") \
     .getOrCreate()

sc = spark.sparkContext 
sc

<SparkContext master=yarn appName=SparkSession>

In [7]:
amazon_data = spark.read.json("s3://qianyielva/product")

In [8]:
amazon_data.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [9]:
amazon_data.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0000000078|  [1, 1]|    5.0|Conversations wit...|08 11, 2004|A3AF8FFZAZYNE5|                null|          Impactful!|    1092182400|
|0000000116|  [5, 5]|    4.0|Interesting Grish...|04 27, 2002| AH2L9G3DQHHAJ|               chris|  Show me the money!|    1019865600|
|0000000116|  [0, 0]|    1.0|The thumbnail is ...|03 24, 2014|A2IIIDRK3PRRZY|              Helene|Listing is all sc...|    1395619200|
|0000000868|[10, 10]|    4.0|I'll be honest. I...|09 11, 2002|A1TADCM7YWPQ8M|            Joel@AWS|Not a Bad Transla...|    1031702400|
|0000013714|  [0, 0]|    4.0|It had all the so...|10 31

In [22]:
amazon_data.cache()

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint]

In [10]:
amazon_data.count()

82677139

In [11]:
amazon_game = spark.read.json("s3://qianyielva/video")

In [12]:
amazon_game.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0700099867| [8, 12]|    1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN|                 123|Pay to unlock con...|    1341792000|
|0700099867|  [0, 0]|    4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...|     Good rally game|    1372550400|
|0700099867|  [0, 0]|    1.0|1st shipment rece...|06 28, 2014|A1INA0F5CWW3J4|Amazon Shopper "M...|           Wrong key|    1403913600|
|0700099867| [7, 10]|    3.0|I got this versio...|09 14, 2011|A1DLMTOTHQ4AST|            ampgreen|awesome game, if ...|    1315958400|
|0700099867|  [2, 2]|    4.0|I had Dirt 2 on X...|06 14

In [19]:
amazon_data.createOrReplaceTempView("amazon_fulldata")

In [20]:
partial = spark.sql("SELECT asin FROM amazon_fulldata WHERE asin = '0700099867'")

In [21]:
partial.show()

+----------+
|      asin|
+----------+
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
|0700099867|
+----------+
only showing top 20 rows



In [24]:
partial.count()

79

In [25]:
amazon_game.createOrReplaceTempView("amazon_gamedata")

In [26]:
partial2 = spark.sql("SELECT asin FROM amazon_gamedata WHERE asin = '0700099867'")
partial2.count()

25

In [27]:
Vasin = amazon_game.select('asin').rdd.flatMap(lambda x: x).collect()

In [64]:
unique = list(set(Vasin))
#unique

In [49]:
from pyspark.sql import Row

In [53]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [54]:
rdd = sc.parallelize(unique)
videogame_id = rdd.map(lambda x: Row(id=unique[0]))
videogame_id = sqlContext.createDataFrame(videogame_id)

In [66]:
videogame_id.createOrReplaceTempView("videogame_id")
videogame_id.count()

10672

In [58]:
amazon_fullgame = spark.sql("SELECT * FROM amazon_fulldata WHERE asin in videogame_id")

ParseException: "\nmismatched input 'FROM' expecting <EOF>(line 1, pos 9)\n\n== SQL ==\nSELECT * FROM amazon_fulldata WHERE asin in videogame_id\n---------^^^\n"